In [3]:
import json

from nsga_iii.graph_nsga_iii import *
from nsga_iii.function_nsga_iii import *
from nsga_iii.population_nsga_iii import *
from nsga_iii.evole_nsga_iii import *
from nsga_iii.individual_nsga_iii import *

def read_data(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

def run_nsga_ii(data_path,result_store, population_size, generations, mutation_rate, crossover_rate, run_time_def):
    data = read_data(data_path)
    nodes = data['nodes']
    servers = data['server']
    clients = data['client']
    edges = data['edges']
    adj_matrix = [[] for _ in range(len(nodes))]
    for edge in edges:
        adj_matrix[edge[0]].append(edge[1])
        adj_matrix[edge[1]].append(edge[0])
    graph = Graph(len(nodes), len(clients), len(servers), len(clients), clients, servers, adj_matrix)
    func = Function()
    scenario_list = data['scenario']
    result = {}
    run_time = 1
    for scenario in scenario_list:
        sum_delay_topo = 0
        delay_update = scenario["delay"]
        for delay in delay_update:
            sum_delay_topo  = sum_delay_topo + delay[2]
        bandwidth_update = scenario["bandwidth"]
        loss_update = scenario["loss"]
        request = scenario["request"]

        graph.updateGraph(delay_update, loss_update, bandwidth_update)

        pop = Population()
        pop.generate_population(graph, func, population_size, len(request), request)
        evole = Evolutionary()
        solutions = evole.evolve(pop, func, graph, generations, population_size, mutation_rate, crossover_rate, 2, 10)
        # print(solutions)
        Pareto = []

        for indi in solutions:
            objective = []
            objective.append(indi.objectives[0]/sum_delay_topo)
            objective.append(1+ indi.objectives[1]/200)
            objective.append(indi.objectives[2])
            Pareto.append(objective)
        result[run_time] = Pareto
        run_time = run_time + 1
        if run_time == run_time_def + 1:
            break
    with open(result_store, 'w') as f:
        json.dump(result, f)

In [4]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\oxford_2_server_2.json', 100, 100, 0.1, 0.8, 50)

In [5]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\oxford_5_server_2.json', 100, 100, 0.1, 0.8, 50)

In [6]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\oxford_15_server_2.json', 100, 100, 0.1, 0.8, 50)

In [7]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\chinanet_2_server_2.json', 100, 100, 0.1, 0.8, 50)

In [8]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\chinanet_5_server_2.json', 100, 100, 0.1, 0.8, 50)

In [9]:
run_nsga_ii(r"data\oxford\2_server.json",r'result\nsga_iii\chinanet_15_server_2.json', 100, 100, 0.1, 0.8, 50)